In [1]:
import pandas as pd
import numpy as np
import os
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
import xgboost as xgb
import lightgbm as lgb
import catboost as cb

# Create column names: X1-X16 for first 16 cols, Y1-Y16 for next 16 cols, power columns for rest
column_names = [f"X{i}" for i in range(1, 17)] + [f"Y{i}" for i in range(1, 17)] + [f"Power{i}" for i in range(1, 17)] + ["Powerall"]

adelaide_df = pd.read_csv('energy/Adelaide_Data.csv', header=None, names=column_names)
perth_df  = pd.read_csv('energy/Perth_Data.csv', header=None, names=column_names)
sydney_df = pd.read_csv('energy/Sydney_Data.csv', header=None, names=column_names)
tasmania_df = pd.read_csv('energy/Tasmania_Data.csv', header=None, names=column_names)

In [2]:
#check if dataset has any wrong values
print("--- Adelaide DataFrame Info ---")
adelaide_df.info()
print("\n--- Adelaide DataFrame Null Counts ---")
print(adelaide_df.isnull().sum())
print(f"\n--- Adelaide DataFrame Duplicate Rows: {adelaide_df.duplicated().sum()} ---")
print("\n--- Adelaide DataFrame Descriptive Statistics ---")
display(adelaide_df.describe())

print("\n\n--- Perth DataFrame Info ---")
perth_df.info()
print("\n--- Perth DataFrame Null Counts ---")
print(perth_df.isnull().sum())
print(f"\n--- Perth DataFrame Duplicate Rows: {perth_df.duplicated().sum()} ---")
print("\n--- Perth DataFrame Descriptive Statistics ---")
display(perth_df.describe())

print("\n\n--- Sydney DataFrame Info ---")
sydney_df.info()
print("\n--- Sydney DataFrame Null Counts ---")
print(sydney_df.isnull().sum())
print(f"\n--- Sydney DataFrame Duplicate Rows: {sydney_df.duplicated().sum()} ---")
print("\n--- Sydney DataFrame Descriptive Statistics ---")
display(sydney_df.describe())

print("\n\n--- Tasmania DataFrame Info ---")
tasmania_df.info()
print("\n--- Tasmania DataFrame Null Counts ---")
print(tasmania_df.isnull().sum())
print(f"\n--- Tasmania DataFrame Duplicate Rows: {tasmania_df.duplicated().sum()} ---")
print("\n--- Tasmania DataFrame Descriptive Statistics ---")
display(tasmania_df.describe())


--- Adelaide DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71999 entries, 0 to 71998
Data columns (total 49 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   X1        71999 non-null  float64
 1   X2        71999 non-null  float64
 2   X3        71999 non-null  float64
 3   X4        71999 non-null  float64
 4   X5        71999 non-null  float64
 5   X6        71999 non-null  float64
 6   X7        71999 non-null  float64
 7   X8        71999 non-null  float64
 8   X9        71999 non-null  float64
 9   X10       71999 non-null  float64
 10  X11       71999 non-null  float64
 11  X12       71999 non-null  float64
 12  X13       71999 non-null  float64
 13  X14       71999 non-null  float64
 14  X15       71999 non-null  float64
 15  X16       71999 non-null  float64
 16  Y1        71999 non-null  float64
 17  Y2        71999 non-null  float64
 18  Y3        71999 non-null  float64
 19  Y4        71999 non-null  float64
 

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,Power8,Power9,Power10,Power11,Power12,Power13,Power14,Power15,Power16,Powerall
count,71999.000000,71999.000000,71999.000000,71999.000000,71999.000000,71999.000000,71999.000000,71999.000000,71999.000000,71999.000000,...,71999.000000,71999.000000,71999.000000,71999.000000,71999.000000,71999.000000,71999.000000,71999.000000,71999.000000,7.199900e+04
mean,281.278924,279.316030,294.136617,263.824010,290.186913,246.810469,252.476041,322.610209,280.743361,288.407746,...,88436.529876,88332.518976,87564.244287,88660.640939,88424.979052,87185.488018,87703.940233,89191.145040,88471.467381,1.410073e+06
std,178.319540,178.036825,182.137730,194.870821,179.637096,191.377291,178.339336,178.435581,190.275108,178.158506,...,10108.670480,10156.721862,10174.872639,10515.790588,10489.965853,10565.601127,10430.102374,10442.279774,10572.633956,5.600730e+04
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,51444.747000,51304.021600,49409.891500,49814.045300,51660.410600,50205.352800,51141.599700,50628.552800,47273.983600,1.191378e+06
25%,117.007350,116.477950,116.269700,76.573300,123.088200,67.495700,89.112800,166.907750,103.621650,126.430400,...,81445.718650,81102.528950,80546.907150,80803.234450,80788.756250,79083.975350,79920.395700,81584.298250,80400.615700,1.371208e+06
50%,282.739600,280.673700,315.465600,247.233200,286.184900,214.922300,233.675600,356.333100,267.170400,288.203900,...,90310.572000,90083.765400,88073.786700,91008.814000,89940.028200,87663.582800,89003.815900,92114.119000,90699.133500,1.402170e+06
75%,437.293800,444.120100,455.152650,444.672800,460.145000,433.310400,408.797000,481.671100,464.014100,448.374300,...,97418.703800,97448.106150,97336.083150,97813.297150,97720.383600,97475.999350,97501.539800,98073.001950,97663.051250,1.446064e+06
max,566.000000,566.000000,566.000000,566.000000,566.000000,566.000000,566.000000,566.000000,566.000000,566.000000,...,105790.374500,105526.299600,105420.107400,105395.279300,105539.986600,105447.760000,105301.202500,104602.318800,105390.327100,1.583052e+06




--- Perth DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72000 entries, 0 to 71999
Data columns (total 49 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   X1        72000 non-null  float64
 1   X2        72000 non-null  float64
 2   X3        72000 non-null  float64
 3   X4        72000 non-null  float64
 4   X5        72000 non-null  float64
 5   X6        72000 non-null  float64
 6   X7        72000 non-null  float64
 7   X8        72000 non-null  float64
 8   X9        72000 non-null  float64
 9   X10       72000 non-null  float64
 10  X11       72000 non-null  float64
 11  X12       72000 non-null  float64
 12  X13       72000 non-null  float64
 13  X14       72000 non-null  float64
 14  X15       72000 non-null  float64
 15  X16       72000 non-null  float64
 16  Y1        72000 non-null  float64
 17  Y2        72000 non-null  float64
 18  Y3        72000 non-null  float64
 19  Y4        72000 non-null  float64
 2

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,Power8,Power9,Power10,Power11,Power12,Power13,Power14,Power15,Power16,Powerall
count,72000.000000,72000.000000,72000.000000,72000.000000,72000.000000,72000.000000,72000.000000,72000.000000,72000.000000,72000.000000,...,72000.000000,72000.000000,72000.000000,72000.000000,72000.000000,72000.000000,72000.000000,72000.000000,72000.000000,7.200000e+04
mean,307.701005,263.069113,310.032429,280.009987,271.573004,271.825573,271.628205,265.366760,300.024726,265.180707,...,87172.791411,87227.400248,87479.702421,87259.608082,86416.545859,86879.938895,86110.179312,88026.263288,87450.093836,1.394475e+06
std,186.406373,182.580380,188.616824,180.758049,183.357609,181.907878,180.591345,182.813160,179.271019,183.327812,...,10574.754639,10196.636050,10087.831693,10470.692266,10574.289769,9942.085387,10371.908246,10471.593551,10477.559853,5.225027e+04
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,46239.804500,50114.929100,47183.047800,49291.778000,49549.990600,49465.685600,47027.625800,48982.438200,46881.573700,1.177711e+06
25%,132.556675,97.074000,126.138925,117.146725,98.519050,102.462975,105.551600,94.703375,134.872000,101.784400,...,79380.352150,79774.712375,79824.967950,79398.137825,78207.534625,80019.021325,78324.401825,80115.251700,79372.788325,1.359069e+06
50%,343.866650,243.846950,347.420450,282.056450,265.243800,267.376650,261.312400,251.311950,316.858700,253.393700,...,88613.666150,88460.835500,88768.455950,88450.972600,87221.242050,87720.772150,86479.809000,90513.687000,89574.041900,1.388878e+06
75%,477.285450,427.381725,480.393700,440.353300,441.640325,435.245675,436.602525,441.454300,461.048600,435.324425,...,96919.859150,96704.497125,96880.945750,96922.366100,96700.623700,96550.188550,96575.292350,97074.480500,96752.089350,1.426946e+06
max,566.000000,566.000000,566.000000,566.000000,566.000000,566.000000,566.000000,566.000000,566.000000,566.000000,...,107228.154900,105509.436700,104693.116900,104978.220800,103852.565800,104757.583200,104435.890300,105785.806400,104430.644700,1.565836e+06




--- Sydney DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72000 entries, 0 to 71999
Data columns (total 49 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   X1        72000 non-null  float64
 1   X2        72000 non-null  float64
 2   X3        72000 non-null  float64
 3   X4        72000 non-null  float64
 4   X5        72000 non-null  float64
 5   X6        72000 non-null  float64
 6   X7        72000 non-null  float64
 7   X8        72000 non-null  float64
 8   X9        72000 non-null  float64
 9   X10       72000 non-null  float64
 10  X11       72000 non-null  float64
 11  X12       72000 non-null  float64
 12  X13       72000 non-null  float64
 13  X14       72000 non-null  float64
 14  X15       72000 non-null  float64
 15  X16       72000 non-null  float64
 16  Y1        72000 non-null  float64
 17  Y2        72000 non-null  float64
 18  Y3        72000 non-null  float64
 19  Y4        72000 non-null  float64
 

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,Power8,Power9,Power10,Power11,Power12,Power13,Power14,Power15,Power16,Powerall
count,72000.000000,72000.000000,72000.000000,72000.000000,72000.000000,72000.000000,72000.000000,72000.000000,72000.000000,72000.000000,...,72000.000000,72000.000000,72000.000000,72000.000000,72000.000000,72000.000000,72000.000000,72000.000000,72000.000000,7.200000e+04
mean,324.130650,318.476619,248.650127,273.804973,364.120788,270.715581,268.090715,251.211585,257.469299,263.833769,...,92460.889647,92400.867081,92359.874711,93528.558491,92533.020595,92866.782188,94515.422258,92124.902168,92812.671388,1.486229e+06
std,201.042023,200.450328,207.727345,202.496658,165.250467,221.881965,210.125347,191.412032,193.920143,203.080397,...,6351.754870,7314.166192,6617.035471,7108.750555,6631.046214,7179.076034,6828.345629,6978.090343,7128.152117,2.308364e+04
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,67649.721100,67935.571500,67961.501700,67930.722800,68171.326100,68113.928900,68294.961800,66233.342800,64609.768500,1.361962e+06
25%,137.449825,131.146525,43.281500,63.622900,248.586900,38.906525,57.463400,79.835250,71.295925,48.945475,...,87871.425850,86957.349700,87505.072375,88246.974000,87321.966400,87602.965375,90044.975600,86835.587700,87872.906700,1.470987e+06
50%,355.491700,355.171500,201.883500,274.834600,388.372100,250.374600,233.006700,213.824750,239.241300,265.379750,...,92484.554050,91374.237650,92113.287250,93441.473900,93067.305700,91552.433900,94274.489100,91203.030000,91453.725900,1.487282e+06
75%,524.001500,518.242925,462.505600,468.289225,515.557000,511.209300,485.902425,422.665350,432.784500,457.983675,...,97638.051900,97719.189525,97502.161300,98653.893875,97534.197050,98739.563075,99668.561125,97512.515500,98362.513050,1.504180e+06
max,566.000000,566.000000,566.000000,566.000000,566.000000,566.000000,566.000000,566.000000,566.000000,566.000000,...,110026.753900,109417.620000,109240.703000,109668.448700,109185.088800,109898.206000,109237.388800,109087.161400,109253.606800,1.536347e+06




--- Tasmania DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72000 entries, 0 to 71999
Data columns (total 49 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   X1        72000 non-null  float64
 1   X2        72000 non-null  float64
 2   X3        72000 non-null  float64
 3   X4        72000 non-null  float64
 4   X5        72000 non-null  float64
 5   X6        72000 non-null  float64
 6   X7        72000 non-null  float64
 7   X8        72000 non-null  float64
 8   X9        72000 non-null  float64
 9   X10       72000 non-null  float64
 10  X11       72000 non-null  float64
 11  X12       72000 non-null  float64
 12  X13       72000 non-null  float64
 13  X14       72000 non-null  float64
 14  X15       72000 non-null  float64
 15  X16       72000 non-null  float64
 16  Y1        72000 non-null  float64
 17  Y2        72000 non-null  float64
 18  Y3        72000 non-null  float64
 19  Y4        72000 non-null  float64

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,Power8,Power9,Power10,Power11,Power12,Power13,Power14,Power15,Power16,Powerall
count,72000.000000,72000.000000,72000.000000,72000.000000,72000.000000,72000.000000,72000.000000,72000.000000,72000.000000,72000.000000,...,72000.000000,72000.000000,72000.000000,72000.000000,72000.000000,72000.00000,72000.000000,72000.000000,72000.000000,7.200000e+04
mean,300.334674,294.454049,289.066287,263.393313,307.688097,246.713427,278.580047,286.571972,276.475900,299.831906,...,232383.809106,235201.221458,239331.914935,235166.905914,234747.645220,233790.87416,235308.969514,238324.695375,236811.563240,3.760135e+06
std,191.473699,178.334355,180.893467,181.188155,192.543076,182.996670,182.148218,169.949660,179.503873,185.270428,...,28444.373453,29019.822451,28813.403078,28734.076347,28698.375147,28354.13899,28499.868118,28694.132104,28045.641350,1.121467e+05
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,129775.910100,138563.996300,126897.265500,133149.911400,128318.157900,131102.36320,134838.948900,135212.637500,128026.083200,3.235131e+06
25%,121.019925,137.409450,126.750025,104.923275,125.937475,75.863775,114.668750,143.879750,122.933775,132.523025,...,211195.885800,213104.082425,217772.123950,213166.245800,212845.370700,211827.84055,213768.764875,217256.762500,216084.572525,3.685857e+06
50%,316.564700,303.406700,301.635700,244.313250,336.138900,223.478250,264.086350,297.624550,261.173500,314.038200,...,233918.815250,236770.006100,245712.065000,237441.053000,237238.161500,236434.52870,237548.666400,242743.065400,240613.484050,3.755820e+06
75%,479.768225,447.732125,446.484425,427.202575,484.349675,407.305150,451.786725,420.962525,442.215300,471.073825,...,259325.638375,265307.618750,265738.635350,264954.393975,263741.435950,261343.41530,264541.756425,265748.740125,265161.642500,3.830819e+06
max,566.000000,566.000000,566.000000,566.000000,566.000000,566.000000,566.000000,566.000000,566.000000,566.000000,...,282143.804400,279824.753000,283548.559400,281457.365400,281789.422200,281968.79520,286279.149600,282189.733800,283875.584200,4.241838e+06


In [3]:
results = {}

# cols 1-16 represent X coordinates
# cols 17-32 represent Y coordinates
# cols 33 - 48 represent power of single containers
# col 49 represents sum of power in the farm
# We want to calculate Powerall depending on X, Y coordinates

X_cols = [f"X{i}" for i in range(1, 17)]
Y_cols = [f"Y{i}" for i in range(1, 17)]
powerall_col = ["Powerall"]

def split_data(subset, powerall):
    X_train, X_temp, y_train, y_temp = train_test_split(subset, powerall, test_size=0.4, random_state=44)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=44)
    return X_train, X_val, X_test, y_train, y_val, y_test


def prepare_subsets(dataframe):
    subset = dataframe[X_cols + Y_cols]
    powerall = dataframe[powerall_col]
    return split_data(subset, powerall)

In [4]:
def plot_predicted_vs_actual(y_test, y_predict, name, r2, rmse, mape, show_plot=False):
    """
    Generates a scatter plot of predicted vs. actual values.
    
    y_test: Actual values
    y_predict: Predicted values
    title: Title for the plot
    """
    plt.figure(figsize=(8, 6))
    plt.scatter(y_test, y_predict, alpha=0.5, label='Predicted vs Actual')
   
    # Ideal line (y=x)
    min_val = min(y_test.min().iloc[0], y_predict.min())
    max_val = max(y_test.max().iloc[0], y_predict.max())
    plt.plot([min_val, max_val], [min_val, max_val], 'k--', lw=2, label='Ideal Fit (y=x)')
   
    plt.xlabel("Actual Values")
    plt.ylabel("Predicted Values")
    plt.title(f"Predicted vs Actual for {name}")
    plt.legend()
    plt.grid(True)

    # Display R2, RMSE, and MAPE values on the plot
    textstr = f"R2: {r2:.4f}\nRMSE: {rmse:.2f}\nMAPE: {mape:.2f}%"
    plt.gca().text(0.05, 0.95, textstr, transform=plt.gca().transAxes, fontsize=10,
                   verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.5))
    
    if show_plot:
        plt.show()
    plots_dir = 'plots/predicted_vs_actualy'
    if not os.path.exists(plots_dir):
        os.makedirs(plots_dir)
   
    filename = name.replace(' ', '_') + '.png'
    filepath = os.path.join(plots_dir, filename)
   
    plt.savefig(filepath)
    plt.close()


def plot_residuals(y_test, y_predict, name,  r2, rmse, mape, show_plot=False):
    """
    Generates a scatter plot of residuals vs. predicted values.
       
    y_test: Actual values
    y_predict: Predicted values
    title: Title for the plot
    name: Name for saving the file
    """
    residuals = y_test.values.flatten() - y_predict.flatten()
        
    plt.figure(figsize=(8, 6))
    plt.scatter(y_predict, residuals, alpha=0.5)
        
    # Horizontal line at y=0
    plt.axhline(y=0, color='k', linestyle='--', lw=2)
        
    plt.xlabel("Predicted Values")
    plt.ylabel("Residuals (Actual - Predicted)")
    plt.title(f"Residuals vs Predicted Values for {name}")
    plt.grid(True)

    # Display R2, RMSE, and MAPE values on the plot
    textstr = f"R2: {r2:.4f}\nRMSE: {rmse:.2f}\nMAPE: {mape:.2f}%"
    plt.gca().text(0.05, 0.95, textstr, transform=plt.gca().transAxes, fontsize=10,
                   verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.5))
    
        
    if show_plot:
        plt.show()
        
    plots_dir = 'plots/residuals'
    if not os.path.exists(plots_dir):
        os.makedirs(plots_dir)
        
    filename = name.replace(' ', '_') + '_residuals.png'
    filepath = os.path.join(plots_dir, filename)
        
    plt.savefig(filepath)
    plt.close()

def plot_residuals_distribution(y_test, y_predict, name,  r2, rmse, mape, show_plot=False):
    """
    Generates a histogram of the residuals to show their distribution.
        
    y_test: Actual values
    y_predict: Predicted values
    title: Title for the plot
    name: Name for saving the file
    """
    residuals = y_test.values.flatten() - y_predict.flatten()
        
    plt.figure(figsize=(8, 6))
        
    plt.hist(residuals, bins=50, density=True, alpha=0.6, color='g', label='Residuals Histogram')
        
    # Fit a normal distribution to the data
    mu, std = stats.norm.fit(residuals)
    xmin, xmax = plt.xlim()
    x = np.linspace(xmin, xmax, 100)
    p = stats.norm.pdf(x, mu, std)
    plt.plot(x, p, 'k', linewidth=2, label='Normal Distribution Fit')
        
    plt.axvline(x=0, color='r', linestyle='--', lw=2, label='Zero Residuals')

    plt.xlabel("Residuals")
    plt.ylabel("Density")
    plt.title(f"Residuals Distribution for {name}")
    plt.legend()
    plt.grid(True)

    # Display R2, RMSE, and MAPE values on the plot
    textstr = f"R2: {r2:.4f}\nRMSE: {rmse:.2f}\nMAPE: {mape:.2f}%"
    plt.gca().text(0.05, 0.95, textstr, transform=plt.gca().transAxes, fontsize=10,
                   verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.5))
        
    if show_plot:
        plt.show()
            
    plots_dir = 'plots/residuals_distribution'
    if not os.path.exists(plots_dir):
        os.makedirs(plots_dir)
            
    filename = name.replace(' ', '_') + '_residuals_dist.png'
    filepath = os.path.join(plots_dir, filename)
            
    plt.savefig(filepath)
    plt.close()

def make_plots(y_test, y_predict, name, r2, rmse, mape):
    plot_predicted_vs_actual(y_test, y_predict, name, r2, rmse, mape)
    plot_residuals(y_test, y_predict, name, r2, rmse, mape)
    plot_residuals_distribution(y_test, y_predict, name, r2, rmse, mape)

    




In [5]:
def train_poly_regression(dataframe, degree, name):
    
    X_train, X_val, X_test, y_train, y_val, y_test = prepare_subsets(dataframe)

    poly_features = PolynomialFeatures(degree=degree, include_bias=False)
    X_train_poly = poly_features.fit_transform(X_train)
    X_test_poly = poly_features.transform(X_test)

    model = LinearRegression()
    
    model.fit(X_train_poly, y_train)
    y_predict = model.predict(X_test_poly)
    r2 = model.score(X_test_poly, y_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_predict))
    mape = mean_absolute_percentage_error(y_test, y_predict) * 100
    print(f"{name}\nScore: {r2}")
    print(f"RMSE: {rmse:.3f}")
    print(f"MAPE: {mape:.3f}%\n")

    results[name] = {
       "score": r2,
       "rmse": rmse,
       "mape": mape
    }

    make_plots(y_test, y_predict, name, r2, rmse, mape)

In [6]:
train_poly_regression(adelaide_df, 1, "Adelaide - Linear Regression")
train_poly_regression(perth_df, 1, "Perth - Linear Regression")
train_poly_regression(sydney_df, 1, "Sydney - Linear Regression")
train_poly_regression(tasmania_df, 1, "Tasmania - Linear Regression")

train_poly_regression(adelaide_df, 2, "Adelaide - Polynomial Regression Degree 2")
train_poly_regression(perth_df, 2, "Perth - Polynomial Regression Degree 2")
train_poly_regression(sydney_df, 2, "Sydney - Polynomial Regression Degree 2")
train_poly_regression(tasmania_df, 2, "Tasmania - Polynomial Regression Degree 2")



Adelaide - Linear Regression
Score: 0.1745709501418038
RMSE: 51144.787
MAPE: 2.906%

Perth - Linear Regression
Score: 0.14944002642095067
RMSE: 48611.310
MAPE: 2.751%

Sydney - Linear Regression
Score: 0.13260959255609428
RMSE: 21606.139
MAPE: 1.169%

Tasmania - Linear Regression
Score: 0.14371569409226093
RMSE: 104221.734
MAPE: 2.169%

Adelaide - Polynomial Regression Degree 2
Score: 0.8455546577463555
RMSE: 22123.249
MAPE: 1.245%

Perth - Polynomial Regression Degree 2
Score: 0.8185148398339247
RMSE: 22454.605
MAPE: 1.263%

Sydney - Polynomial Regression Degree 2
Score: 0.8247720650661557
RMSE: 9711.169
MAPE: 0.485%

Tasmania - Polynomial Regression Degree 2
Score: 0.7297021342418895
RMSE: 58555.893
MAPE: 1.217%



In [7]:
import xgboost as xgb

def train_xgb_model(dataframe, name):

    subset = dataframe[X_cols + Y_cols]
    powerall = dataframe[powerall_col]


    X_train, X_test, y_train, y_test = train_test_split(subset, powerall, test_size=0.2, random_state=44)

    xgb_model = xgb.XGBRegressor (objective="reg:squarederror", n_estimators = 1500, random_state=44, subsample= 0.8, learning_rate= 0.07)
    xgb_model.fit(X_train, y_train)
    y_predict = xgb_model.predict(X_test)

    r2 = r2_score(y_test, y_predict)
    rmse = np.sqrt(mean_squared_error(y_test, y_predict))
    mape = mean_absolute_percentage_error(y_test, y_predict) * 100

    print(f"{name}\nScore: {r2}")
    print(f"RMSE: {rmse:.3f}")
    print(f"MAPE: {mape:.3f}%\n")

    results[name] = {
        "score": r2,
        "rmse": rmse,
        "mape": mape
        }
    
    make_plots(y_test, y_predict, name, r2, rmse, mape)

In [ ]:
train_xgb_model(adelaide_df, "Adelaide - XGBoost Regressor")
train_xgb_model(perth_df, "Perth - XGBoost Regressor")
train_xgb_model(sydney_df, "Sydney - XGBoost Regressor")
train_xgb_model(tasmania_df, "Tasmania - XGBoost Regressor")

In [ ]:
import lightgbm as lgb

def train_lgb_model(dataframe, name):

    subset = dataframe[X_cols + Y_cols]
    powerall = dataframe[powerall_col]

    X_train, X_test, y_train, y_test = train_test_split(subset, powerall, test_size=0.2, random_state=44)

    lgb_model = lgb.LGBMRegressor(
        objective="regression", 
        n_estimators=1500, 
        random_state=44, 
        subsample=0.8, 
        learning_rate=0.07
    )
    
    lgb_model.fit(X_train, y_train)
    y_predict = lgb_model.predict(X_test)

    r2 = r2_score(y_test, y_predict)
    rmse = np.sqrt(mean_squared_error(y_test, y_predict))
    mape = mean_absolute_percentage_error(y_test, y_predict) * 100

    print(f"{name}\nScore: {r2}")
    print(f"RMSE: {rmse:.3f}")
    print(f"MAPE: {mape:.3f}%\n")

    results[name] = {
        "score": r2,
        "rmse": rmse,
        "mape": mape
    }

    make_plots(y_test, y_predict, name, r2, rmse, mape)

In [ ]:
train_lgb_model(adelaide_df, "Adelaide - LightGBM Regressor")
train_lgb_model(perth_df, "Perth - LightGBM Regressor")
train_lgb_model(sydney_df, "Sydney - LightGBM Regressor")
train_lgb_model(tasmania_df, "Tasmania - LightGBM Regressor")

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002987 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8160
[LightGBM] [Info] Number of data points in the train set: 57599, number of used features: 32
[LightGBM] [Info] Start training from score 1410128.507641
Adelaide - LightGBM Regressor
Score: 0.9017606010861957
RMSE: 17548.033
MAPE: 0.935%

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005468 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8160
[LightGBM] [Info] Number of data points in the train set: 57600, number of used features: 32
[LightGBM] [Info] Start training from score 1394560.146675
Perth - LightGBM Regressor
Score: 0.8813326497717089
RMSE: 17930.987
MAPE: 0.974%

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005103 seconds.
You can set `force_col_wise=tru

In [ ]:
import catboost as cb

def train_catboost_model(dataframe, name):
    X_cols = [f"X{i}" for i in range(1, 17)]
    Y_cols = [f"Y{i}" for i in range(1, 17)]
    powerall_col = ["Powerall"]
    subset = dataframe[X_cols + Y_cols]
    powerall = dataframe[powerall_col]

    X_train, X_temp, y_train, y_temp = train_test_split(subset, powerall, test_size=0.4, random_state=44)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=44)

    catboost_model = cb.CatBoostRegressor(
        iterations=3000,
        learning_rate=0.05,
        depth=7,
        l2_leaf_reg=3,
        random_seed=44,
        verbose=False,
        early_stopping_rounds=100
    )
    
    catboost_model.fit(
        X_train, 
        y_train,
        eval_set=(X_val, y_val),
        verbose=False
    )
    
    y_predict = catboost_model.predict(X_test)

    r2 = r2_score(y_test, y_predict)
    rmse = np.sqrt(mean_squared_error(y_test, y_predict))
    mape = mean_absolute_percentage_error(y_test, y_predict) * 100

    print(f"{name}\nScore: {r2:.4f}")
    print(f"RMSE: {rmse:.3f}")
    print(f"MAPE: {mape:.3f}%\n")

    results[name] = {
        "score": r2,
        "rmse": rmse,
        "mape": mape
    }

    make_plots(y_test, y_predict, name, r2, rmse, mape)

In [ ]:
train_catboost_model(adelaide_df, "Adelaide - CatBoost Regressor")
train_catboost_model(perth_df, "Perth - CatBoost Regressor")
train_catboost_model(sydney_df, "Sydney - CatBoost Regressor")
train_catboost_model(tasmania_df, "Tasmania - CatBoost Regressor")

Adelaide - CatBoost Regressor
Score: 0.9090
RMSE: 16978.547
MAPE: 0.896%

Perth - CatBoost Regressor
Score: 0.8886
RMSE: 17595.356
MAPE: 0.949%

Sydney - CatBoost Regressor
Score: 0.9224
RMSE: 6462.188
MAPE: 0.297%

Tasmania - CatBoost Regressor
Score: 0.8338
RMSE: 45916.585
MAPE: 0.934%



In [ ]:
def train_ensemble_model(dataframe, name):
    X_cols = [f"X{i}" for i in range(1, 17)]
    Y_cols = [f"Y{i}" for i in range(1, 17)]
    powerall_col = ["Powerall"]
    subset = dataframe[X_cols + Y_cols]
    powerall = dataframe[powerall_col]

    X_train, X_temp, y_train, y_temp = train_test_split(subset, powerall, test_size=0.4, random_state=44)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=44)


    xgb_model = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=3000, 
                                  learning_rate=0.05, max_depth=7, subsample=0.8, 
                                  random_state=44, colsample_bytree=0.8)
    
    lgb_model = lgb.LGBMRegressor(objective="regression", n_estimators=3000, 
                                   learning_rate=0.05, max_depth=7, subsample=0.8, 
                                   random_state=44, colsample_bytree=0.8, verbose=-1)
    
    catboost_model = cb.CatBoostRegressor(iterations=3000, learning_rate=0.05, 
                                          depth=7, random_seed=44, verbose=False)

 
    xgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    lgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], 
                  callbacks=[lgb.early_stopping(stopping_rounds=100)])
    catboost_model.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=False)


    pred_xgb = xgb_model.predict(X_test)
    pred_lgb = lgb_model.predict(X_test)
    pred_cat = catboost_model.predict(X_test)


    y_predict = 0.36 * pred_xgb + 0.36 * pred_lgb + 0.28 * pred_cat

    r2 = r2_score(y_test, y_predict)
    rmse = np.sqrt(mean_squared_error(y_test, y_predict))
    mape = mean_absolute_percentage_error(y_test, y_predict) * 100

    print(f"{name}\nScore: {r2:.4f}")
    print(f"RMSE: {rmse:.3f}")
    print(f"MAPE: {mape:.3f}%\n")

    results[name] = {
        "score": r2,
        "rmse": rmse,
        "mape": mape
    }

    make_plots(y_test, y_predict, name, r2, rmse, mape)


In [ ]:
train_ensemble_model(adelaide_df, "Adelaide - Ensemble (XGB+LGB+Cat)")
train_ensemble_model(perth_df, "Perth - Ensemble (XGB+LGB+Cat)")
train_ensemble_model(sydney_df, "Sydney - Ensemble (XGB+LGB+Cat)")
train_ensemble_model(tasmania_df, "Tasmania - Ensemble (XGB+LGB+Cat)")

Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[3000]	valid_0's l2: 2.94498e+08
Adelaide - Ensemble (XGB+LGB+Cat)
Score: 0.9104
RMSE: 16847.497
MAPE: 0.885%

Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[2997]	valid_0's l2: 3.18993e+08
Perth - Ensemble (XGB+LGB+Cat)
Score: 0.8896
RMSE: 17510.181
MAPE: 0.938%

Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[3000]	valid_0's l2: 4.29638e+07
Sydney - Ensemble (XGB+LGB+Cat)
Score: 0.9271
RMSE: 6264.027
MAPE: 0.265%

Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[2999]	valid_0's l2: 2.14339e+09
Tasmania - Ensemble (XGB+LGB+Cat)
Score: 0.8347
RMSE: 45795.159
MAPE: 0.927%



In [ ]:
for name, metrics in results.items():
    print(f"\n{name}:")
    for metric, value in metrics.items():
        print(f"  {metric}: {value}")


Adelaide - Linear Regression:
  score: 0.1745709501418038
  rmse: 51144.786579729225
  mape: 2.9064190651831594

Perth - Linear Regression:
  score: 0.14944002642095067
  rmse: 48611.30996211733
  mape: 2.7505659369864275

Sydney - Linear Regression:
  score: 0.13260959255609428
  rmse: 21606.138651417088
  mape: 1.1691562311047736

Tasmania - Linear Regression:
  score: 0.14371569409226093
  rmse: 104221.73408520623
  mape: 2.1690888425474233

Adelaide - Polynomial Regression Degree 2:
  score: 0.8455546577463555
  rmse: 22123.24881646634
  mape: 1.2454148094950672

Perth - Polynomial Regression Degree 2:
  score: 0.8185148398339247
  rmse: 22454.60466406559
  mape: 1.2633278031244077

Sydney - Polynomial Regression Degree 2:
  score: 0.8247720650661557
  rmse: 9711.16923726056
  mape: 0.4848479106144605

Tasmania - Polynomial Regression Degree 2:
  score: 0.7297021342418895
  rmse: 58555.893335135246
  mape: 1.2166544054952246

Adelaide - XGBoost Regressor:
  score: 0.90413308143615

In [ ]:
import optuna
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)



def objective(trial, X_train, y_train, X_val, y_val):

    params = {
        "objective": "reg:squarederror",
        "eval_metric": "rmse",
        "n_estimators": 2000,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "gamma": trial.suggest_float("gamma", 1e-8, 1.0, log=True),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "random_state": 44,
        "n_jobs": -1
    }
    
    model = xgb.XGBRegressor(
        **params,
        early_stopping_rounds=50
    )

    model.fit(
        X_train, 
        y_train, 
        eval_set=[(X_val, y_val)],
        verbose=False
    )

    trial.set_user_attr("best_iteration", model.best_iteration)
    
    
    y_predict = model.predict(X_val)
    
    
    rmse = np.sqrt(mean_squared_error(y_val, y_predict))
    return rmse

def train_xgb_with_optuna(dataframe, name, n_trials=50):
    
    print(f"--- Optuna tuning for: {name} ---")
    
    X_cols = [f"X{i}" for i in range(1, 17)]
    Y_cols = [f"Y{i}" for i in range(1, 17)]
    powerall_col = ["Powerall"]
    subset = dataframe[X_cols + Y_cols]
    powerall = dataframe[powerall_col]


    # 60% train, 40% temp
    X_train, X_temp, y_train, y_temp = train_test_split(subset, powerall, test_size=0.4, random_state=44)
    # 50% of temp -> validation
    # 50% of temp -> test
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=44)


    study = optuna.create_study(direction="minimize")
    study.optimize(
        lambda trial: objective(trial, X_train, y_train, X_val, y_val), 
        n_trials=n_trials
    )

    print(f"\n--- Tuning for: {name} ended ---")
    print(f"The best iteration (Validation RMSE): {study.best_value:.3f}")
    print("The best hiperparameters:")
    print(study.best_params)

    best_params = study.best_params
    best_iteration = study.best_trial.user_attrs["best_iteration"]
    
    # Combine train (60%) and validation (20%) sets for final training
    X_train_full = pd.concat([X_train, X_val])
    y_train_full = pd.concat([y_train, y_val])
    
    final_model = xgb.XGBRegressor(
        **best_params,
        n_estimators=best_iteration,
        random_state=44,
        n_jobs=4
    )
    
    final_model.fit(X_train_full, y_train_full)
    
    y_predict = final_model.predict(X_test)
    
    r2 = r2_score(y_test, y_predict)
    rmse = np.sqrt(mean_squared_error(y_test, y_predict))
    mape = mean_absolute_percentage_error(y_test, y_predict) * 100

    print(f"\n--- Results for {name} ---")
    print(f"Score (R2): {r2:.4f}")
    print(f"RMSE: {rmse:.3f}")
    print(f"MAPE: {mape:.3f}%\\n")
    results[f"{name}_Optuna_R2"] = {"score": r2, "rmse": rmse, "mape": mape}
    make_plots(y_test, y_predict, name + " - Optuna Tuning", r2, rmse, mape)

In [ ]:
train_xgb_with_optuna(adelaide_df, "Adelaide - XGBoost with Optuna HPT", n_trials=50)
train_xgb_with_optuna(perth_df, "Perth - XGBoost with Optuna HPT", n_trials=50)
train_xgb_with_optuna(sydney_df, "Sydney - XGBoost with Optuna HPT", n_trials=50)
train_xgb_with_optuna(tasmania_df, "Tasmania - XGBoost with Optuna HPT", n_trials=50)

[I 2025-11-25 19:24:09,179] A new study created in memory with name: no-name-5a75e1cd-3609-4a87-bc3a-4ffe153d6d07


--- Optuna tuning for: Adelaide - XGBoost with Optuna HPT ---


[I 2025-11-25 19:24:22,249] Trial 0 finished with value: 19910.818767695113 and parameters: {'learning_rate': 0.148943211492241, 'max_depth': 9, 'subsample': 0.9235052938044574, 'colsample_bytree': 0.9532706264009774, 'gamma': 0.09286707381119697, 'reg_alpha': 7.516854521316566e-07, 'reg_lambda': 1.1266969160904916e-07}. Best is trial 0 with value: 19910.818767695113.
[I 2025-11-25 19:24:30,043] Trial 1 finished with value: 18096.886804088706 and parameters: {'learning_rate': 0.01521478344333008, 'max_depth': 7, 'subsample': 0.9814707026012683, 'colsample_bytree': 0.8086820149691745, 'gamma': 0.0011704439275789116, 'reg_alpha': 7.58475459698496e-08, 'reg_lambda': 0.0014393412019140916}. Best is trial 1 with value: 18096.886804088706.
[I 2025-11-25 19:24:32,799] Trial 2 finished with value: 17880.78208580374 and parameters: {'learning_rate': 0.1259564093326749, 'max_depth': 4, 'subsample': 0.7984258683767347, 'colsample_bytree': 0.831797284417624, 'gamma': 0.011440338699832754, 'reg_alp


--- Tuning for: Adelaide - XGBoost with Optuna HPT ended ---
The best iteration (Validation RMSE): 17299.491
The best hiperparameters:
{'learning_rate': 0.054661819857065565, 'max_depth': 6, 'subsample': 0.6930187833511031, 'colsample_bytree': 0.8629562804087731, 'gamma': 2.0401212058123916e-05, 'reg_alpha': 0.017587446925232952, 'reg_lambda': 5.795348487278384e-07}

--- Results for Adelaide - XGBoost with Optuna HPT ---
Score (R2): 0.9065
RMSE: 17210.135
MAPE: 0.910%\n


[I 2025-11-25 19:30:15,795] A new study created in memory with name: no-name-5296a051-94e6-419f-8d38-907a66f73f97


--- Optuna tuning for: Perth - XGBoost with Optuna HPT ---


[I 2025-11-25 19:30:18,536] Trial 0 finished with value: 18486.82038642665 and parameters: {'learning_rate': 0.15495368417212518, 'max_depth': 3, 'subsample': 0.7721313792260174, 'colsample_bytree': 0.8622195992891464, 'gamma': 0.037910207213409736, 'reg_alpha': 0.03580079198531317, 'reg_lambda': 5.070606457799454e-07}. Best is trial 0 with value: 18486.82038642665.
[I 2025-11-25 19:30:21,679] Trial 1 finished with value: 18165.12350632387 and parameters: {'learning_rate': 0.05231488259597966, 'max_depth': 4, 'subsample': 0.826854127409946, 'colsample_bytree': 0.7399301744972612, 'gamma': 0.6983729647098038, 'reg_alpha': 0.015768876537286647, 'reg_lambda': 4.7222843340641765}. Best is trial 1 with value: 18165.12350632387.
[I 2025-11-25 19:30:26,602] Trial 2 finished with value: 18963.363414753196 and parameters: {'learning_rate': 0.12843953439647274, 'max_depth': 7, 'subsample': 0.9786960771655163, 'colsample_bytree': 0.7076956272529076, 'gamma': 0.1775775897829326, 'reg_alpha': 2.154

KeyboardInterrupt: 

In [ ]:
def objective_lgb(trial, X_train, y_train, X_val, y_val):
    
    params = {
        "objective": "regression_l2",
        "metric": "rmse",
        "n_estimators": 2000,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 20, 150),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "random_state": 44,
        "n_jobs": -1,
        "verbose": -1
    }

    model = lgb.LGBMRegressor(**params)

    model.fit(
        X_train,
        y_train,
        eval_set=[(X_val, y_val)],
        eval_metric="rmse",
        callbacks=[lgb.early_stopping(50, verbose=False)]
    )

    trial.set_user_attr("best_iteration", model.best_iteration_)
    
    y_predict = model.predict(X_val)
    
    rmse = np.sqrt(mean_squared_error(y_val, y_predict))
    return rmse

def train_lgb_with_optuna(dataframe, name, n_trials=50):
    
    print(f"--- Optuna tuning for: {name} ---")
    
    X_cols = [f"X{i}" for i in range(1, 17)]
    Y_cols = [f"Y{i}" for i in range(1, 17)]
    powerall_col = ["Powerall"]
    subset = dataframe[X_cols + Y_cols]
    powerall = dataframe[powerall_col]

    # 60% train, 40% temp
    X_train, X_temp, y_train, y_temp = train_test_split(subset, powerall, test_size=0.4, random_state=44)
    # 50% of temp -> validation
    # 50% of temp -> test
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=44)


    study = optuna.create_study(direction="minimize")
    study.optimize(
        lambda trial: objective_lgb(trial, X_train, y_train, X_val, y_val), 
        n_trials=n_trials
    )

    print(f"\n--- Tuning for: {name} ended ---")
    print(f"The best iteration (Validation RMSE): {study.best_value:.3f}")
    print("The best hiperparameters:")
    print(study.best_params)

    best_params = study.best_params
    best_iteration = study.best_trial.user_attrs["best_iteration"]
    
    # Combine train (60%) and validation (20%) sets for final training
    X_train_full = pd.concat([X_train, X_val])
    y_train_full = pd.concat([y_train, y_val])
    
    final_model = lgb.LGBMRegressor(
        **best_params,
        n_estimators=best_iteration,
        random_state=44,
        n_jobs=-1,
        verbose=-1
    )
    
    final_model.fit(X_train_full, y_train_full)
    
    y_predict = final_model.predict(X_test)
    
    r2 = r2_score(y_test, y_predict)
    rmse = np.sqrt(mean_squared_error(y_test, y_predict))
    mape = mean_absolute_percentage_error(y_test, y_predict) * 100

    print(f"\n--- Results for {name} ---")
    print(f"Score (R2): {r2:.4f}")
    print(f"RMSE: {rmse:.3f}")
    print(f"MAPE: {mape:.3f}%\\n")
    results[f"{name}_Optuna_LGB"] = {"score": r2, "rmse": rmse, "mape": mape}
    make_plots(y_test, y_predict, name + " - Optuna Tuning", r2, rmse, mape)

In [ ]:
train_lgb_with_optuna(adelaide_df, "Adelaide - LightGBM with Optuna HPT", n_trials=50)
train_lgb_with_optuna(perth_df, "Perth - LightGBM with Optuna HPT", n_trials=50)
train_lgb_with_optuna(sydney_df, "Sydney - LightGBM with Optuna HPT", n_trials=50)
train_lgb_with_optuna(tasmania_df, "Tasmania - LightGBM with Optuna HPT", n_trials=50)

In [ ]:
import catboost as cb

def objective_cb(trial, X_train, y_train, X_val, y_val):
    
    params = {
        "objective": "RMSE",
        "eval_metric": "RMSE",
        "iterations": 2000,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "depth": trial.suggest_int("depth", 3, 10),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-8, 10.0, log=True),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "random_strength": trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
        "random_seed": 44,
        "thread_count": -1,
        "verbose": False
    }

    model = cb.CatBoostRegressor(
        **params,
        early_stopping_rounds=50
    )

    model.fit(
        X_train,
        y_train,
        eval_set=(X_val, y_val),
        verbose=False
    )

    trial.set_user_attr("best_iteration", model.get_best_iteration())
    
    y_predict = model.predict(X_val)
    
    rmse = np.sqrt(mean_squared_error(y_val, y_predict))
    return rmse

def train_cb_with_optuna(dataframe, name, n_trials=50):
    
    print(f"--- Optuna tuning for: {name} ---")
    
    X_cols = [f"X{i}" for i in range(1, 17)]
    Y_cols = [f"Y{i}" for i in range(1, 17)]
    powerall_col = ["Powerall"]
    subset = dataframe[X_cols + Y_cols]
    powerall = dataframe[powerall_col]

    # 60% train, 40% temp
    X_train, X_temp, y_train, y_temp = train_test_split(subset, powerall, test_size=0.4, random_state=44)
    # 50% of temp -> validation
    # 50% of temp -> test
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=44)


    study = optuna.create_study(direction="minimize")
    study.optimize(
        lambda trial: objective_cb(trial, X_train, y_train, X_val, y_val), 
        n_trials=n_trials
    )

    print(f"\n--- Tuning for: {name} ended ---")
    print(f"The best iteration (Validation RMSE): {study.best_value:.3f}")
    print("The best hiperparameters:")
    print(study.best_params)

    best_params = study.best_params
    best_iteration = study.best_trial.user_attrs["best_iteration"]
    
    # Combine train (60%) and validation (20%) sets for final training
    X_train_full = pd.concat([X_train, X_val])
    y_train_full = pd.concat([y_train, y_val])
    
    final_model = cb.CatBoostRegressor(
        **best_params,
        iterations=best_iteration,
        random_seed=44,
        thread_count=-1,
        verbose=False
    )
    
    final_model.fit(X_train_full, y_train_full, verbose=False)
    
    y_predict = final_model.predict(X_test)
    
    r2 = r2_score(y_test, y_predict)
    rmse = np.sqrt(mean_squared_error(y_test, y_predict))
    mape = mean_absolute_percentage_error(y_test, y_predict) * 100

    print(f"\n--- Results for {name} ---")
    print(f"Score (R2): {r2:.4f}")
    print(f"RMSE: {rmse:.3f}")
    print(f"MAPE: {mape:.3f}%\\n")
    results[f"{name}_Optuna_CB"] = {"score": r2, "rmse": rmse, "mape": mape}
    make_plots(y_test, y_predict, name + " - Optuna Tuning", r2, rmse, mape)

In [ ]:
train_cb_with_optuna(adelaide_df, "Adelaide - CatBoost with Optuna HPT", n_trials=50)

In [ ]:
train_cb_with_optuna(perth_df, "Perth - CatBoost with Optuna HPT", n_trials=50)

In [ ]:
train_cb_with_optuna(sydney_df, "Sydney - CatBoost with Optuna HPT", n_trials=50)

In [ ]:
train_cb_with_optuna(tasmania_df, "Tasmania - CatBoost with Optuna HPT", n_trials=50)

In [ ]:
print(results)